## Conservation Genomics Practical

------------------------ 
**Background**:

The primary data for modern conservation genomics efforts often includes whole-genome
sequences of multiple individuals of a threatened species. From this data, we infer the levels
and patterns of genetic diversity, and apply population genetics models to infer past
demography. We can also infer levels of inbreeding and patterns of migration from sequence
data. If we have access to information about the deleterious consequences of variants, or we
apply computation inference of those deleterious effects, we can further infer
<br>
<br>
I have four sets of sequence data, all drawn from one chromosome, from four different populations with distinct population histories. The data consist of only the segregating sites, with “0” being the ancestral allele, and “1” being the derived allele. Each row represents a haplotype drawn from the population, and each dataset consists of 50 such
haplotypes.
I will randomly select one of the blocks from each data file, and, for each data set, I will:
1. Read in the data into a matrix
2. Calculate the nucleotide diversity, pi (the data are the segregating sites **only**, out of a span of 10 kb).
3. Determine the number of segregating sites, S.
4. Calculate Watterson’s theta.
5. Calculate Tajima’s D

**Equations:** <br>
Watterson's estimator ($\theta$) <br>
&nbsp; $\theta=\frac{S}{\sum \limits_{i=1}^{n-1}1/i}$ &nbsp; ; &nbsp; where $S$=segregating sites, $n$=the number of loci (columns). <br>
<br>
Average nucleotide diversity ($\pi$) <br>
&nbsp; $ \pi=\frac{1}{L} \sum\limits_{k=1}^{L}h_k$ &nbsp; ; &nbsp; where $h_{k}=\frac{2p(n-p)}{n(n-1)}$ , $L$=number of loci (i.e. nucelotide positions / columns), $n$=sample size (number of haplotypes/rows) , and $p$=number of derived alleles (1's) at loci $k$. <br>
<br>
Tajima's $D$ <br>
$D=\frac{\pi-\theta}{SD(\pi-\theta)}$ 

In [20]:
#imports
import numpy as np
import pandas as pd
from itertools import combinations
import random
#from scipy.stats import tajima_d

In [21]:
# Input matrices 
def load_haplotype_data(file_path):
    with open(file_path, 'r') as file:    # 'r' = read mode
        blocks = file.read().split('\n\n')   #split file in to blocks from newlines
        block = random.choice(blocks)
        processed_lines = []
        # from each file, select one block at random,
        for line in block.strip().split('\n'):
            stripped_line = line.strip()
            if stripped_line.replace(",", "").isdigit():
                processed_lines.append(stripped_line.split(','))
        return pd.DataFrame(processed_lines).astype(int) 
    # each block consists of haplotypes from 50 individuals (rows)

# Initialize a list to store each haplotype matrix
haplotype_blocks = []

# Need to do the following for all 4 blocks selected from the 4 files.
#hapmx = pd.read_csv('ConGensData1block1.txt', sep = ',', header=None)
hapmx1 = load_haplotype_data('Data/ConsGenData1.txt')
hapmx2 = load_haplotype_data('Data/ConsGenData2.txt')
hapmx3 = load_haplotype_data('Data/ConsGenData3.txt')
hapmx4 = load_haplotype_data('Data/ConsGenData4.txt')

haplotype_blocks.extend([hapmx1, hapmx2, hapmx3, hapmx4])

print(haplotype_blocks[0]) #display first matrix for reference

    0   1   2   3   4   5   6   7   8   9   ...  50  51  52  53  54  55  56  \
0    0   0   0   0   0   0   0   0   1   0  ...   0   1   0   0   0   0   0   
1    0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
2    1   1   0   1   0   0   0   0   1   0  ...   0   0   0   0   0   1   0   
3    0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
4    1   1   0   1   0   0   0   0   1   0  ...   0   0   0   0   0   1   0   
5    0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
6    0   0   0   0   0   0   0   0   1   0  ...   0   1   0   0   0   0   0   
7    0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
8    0   0   0   0   0   0   0   0   1   0  ...   0   0   0   1   0   0   0   
9    0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
10   0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
11   0   0   0   0   0   0   0   0   0   0  ...   0 

In [22]:
# Calculate nucelotide diversity (pi)
def calc_pi(haplotypes):
    # variables
    n = haplotypes.shape[0]  # number of haplotypes (rows)
    L = haplotypes.shape[1]  # number of loci (columns)
    # calculations
    if L == 0:
        return 0
    pi_total = 0
    for k in range(L):
        p = np.sum(haplotypes.iloc[:, k] == 1)  # number of derived alleles (1's) at loci k
        h_k = 2 * p * (n - p) / (n * (n - 1))
        pi_total += h_k
    
    #formula
    pi = pi_total / L  # avg nucleotide diversity
    return pi

for i, haplotype_block in enumerate(haplotype_blocks):
    result_pi = calc_pi(haplotype_block)
    #edited to look at each block
    print(f"Nucleotide diversity (pi) for block {i+1}: {result_pi}")

Nucleotide diversity (pi) for block 1: 0.19292517006802717
Nucleotide diversity (pi) for block 2: 0
Nucleotide diversity (pi) for block 3: 0.30892128279883363
Nucleotide diversity (pi) for block 4: 0.23289167974882258


In [24]:
# Calculate the number of segregating sites (S)
def count_segregating_sites(haplotypes):
    n = len(haplotypes.axes[1])
    S = 0
    for i in range(0,n):
        if len(haplotypes[i].unique()) == 1: 
            S = S
        else:
            S = S + 1
    return S

for i, hapmx in enumerate(haplotype_blocks):
    result_S = count_segregating_sites(hapmx)
    print(f"Number of segregating sites (S) for block {i+1}: {result_S} out of {haplotype_block.shape[1]} sites.")

NameError: name 'n' is not defined

In [ ]:
# Calculate wattersons theta
def calc_watterson(haplotypes):
    # S = segregating sites
    # a = sum(1/i)
    # pi = S / a ...
    return watt

result_watt = calculate_watterson(hapmx)
print(f"Wattersons theta: {result_watt}")

In [ ]:
# Calculate Tajima's D
def tajimas_d(haplotypes):
    # variable
    # formula
    # calculations
    Tajimas_D = numerator / denominator
    return Tajimas_D

result_D = tajimas_d(haplotype_matrix)
print(f"Tajima's D: {result_D}")